In [11]:
# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from langdetect import detect
import time
from selenium.webdriver import Firefox, Chrome
import pandas as pd
import re

In [ ]:
keyword = "sports"
base_url = "https://amazon.com/"
# Fetch products for specific keyword
def fetch_products_by_keyword(keyword):
    driver = Chrome()
    driver.maximize_window()
    page_number = 0
    while True:
        page_number += 1
        driver.get(base_url + "s?k=" + keyword + "&page=" + str(page_number) + "&ref=nb_sb_noss_2")
        print(base_url + "s?k=" + keyword + "&page=" + str(page_number) + "&ref=nb_sb_noss_2")
        products = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        for product in products:
            with open("products_" + keyword + ".txt", "a") as file:
                file.write(base_url + "dp/" + product.get_attribute("data-asin") + "\n")
        try:
            next_btn = driver.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
            time.sleep(5)
        except Exception as ex:
            print('Finished scraping keyword: ' + keyword)
            break
    driver.quit()
fetch_products_by_keyword(keyword)

In [30]:
# Iterate through products from txt file and fetch reviews for every comment
def get_comments(fileName):
    reviewedon, prodname = [],[]
    file = open(fileName)
    LIST=re.split("\r\n|\n",file.read())
    file.close()
    x = []
    for i in LIST:
        print(i.split('~')[-1])
        x.append(i.split('~')[-1])
    star_filter_on = True
    star_filter_value = 5
    comments = []
    driver = Chrome()
    df = pd.DataFrame()
    driver.maximize_window()
    i = 0
    while i != len(x):

        url = x[i]
        try:
            driver.get(url)
            product_name = url.split("/")[3].replace("-"," ")
            prodname.append(product_name)
        except Exception as ex:
            print(ex)
        try:
            all_reviews = driver.find_element(By.CSS_SELECTOR,'#reviews-medley-footer .a-text-bold')
            all_reviews.click()
            # Check if star filter is on to just get reviews with specific star
            if star_filter_on:
                stars_list = driver.find_element(By.XPATH, '//span[@id="a-autoid-5-announce"]')
                stars_list.click()
                choose_star = driver.find_element(By.XPATH, f'//a[@id="star-count-dropdown_{star_filter_value}"]')
                choose_star.click()
        except Exception as ex:
            print(ex)
        while True:
            lst_of_ratings = driver.find_elements(By.CLASS_NAME, "review-rating")
            review_title = driver.find_elements(By.XPATH, "//a[@data-hook='review-title']/span")
            loc_date = driver.find_elements(By.XPATH, "//span[@data-hook='review-date']")
            review = driver.find_elements(By.XPATH, '//span[@data-hook="review-body"]')
            # reviewer = driver.find_elements(By.CSS_SELECTOR, '#cm_cr-review_list .a-profile-name')
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "#cm_cr-pagination_bar a")
            except:
                pass
            for k in range(len(review_title)):
                try:
                    date = loc_date[k].text.split('on')
                except:
                    pass  
                reviewedon.append(date)
                for value in lst_of_ratings:
                    try:
                        rating = value.get_attribute('textContent').split(" ")[0]
                    except:
                        rating='Not Available!'
                try:
                    if review[k].text:
                        check_lang = detect(review[k].text)
                        if check_lang != 'en':
                            print('Skipping non english comment')
                            pass
                    commentObj = {
                        'review_title': review_title[k].text,
                        'review_comment': review[k].text,
                        'review_date': date,
                        'review_rating': rating
                    }
                    comments.append(commentObj)
                except:
                    pass
            try:
                next_btn = driver.find_element(By.XPATH, "//ul[@class='a-pagination']/li[2]/a")
                next_btn.click()
                time.sleep(1)
            except Exception as ex:
                break
        i+=1
        
    df = pd.DataFrame(comments)
    df.to_csv("data.csv", mode='a', index=False)
    driver.close()
    print("Finished with file: ")

In [ ]:
get_comments('products_controllers.txt')
get_comments('products_playstation.txt')
get_comments('products_video games.txt')
get_comments('products_xbox.txt')
get_comments('products_crafting.txt')
get_comments('products_camera and photo.txt')
get_comments('products_software.txt')
get_comments('products_sports.txt')

In [32]:
# This function will remove duplicate comments from csv file
def clear_duplicate_comments(fileName):
    data = pd.read_csv(fileName)
    result = data.drop_duplicates(['review_comment'],keep='first')
    result.to_csv("data-set-cleared.csv",index=False)
    
clear_duplicate_comments("data-set.csv")